# Cleaning All the Years Datasets
refer back to 2017_data_eda to see the specific details

In [135]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
import os

In [136]:
pharma_comps = pd.read_csv("other datasets/WR-Issued-June-2024-Approved-Active-Moiety-6-27-24.csv")

In [137]:
#petitions_2013_raw = pd.read_csv("need_to_be_cleaned_datasets/2013_PETITIONS_COMPLETE_050125_WZ_0725_needclean.csv")
def combine_date_columns(df, date_col1, date_col2, new_col="Date of Petition",
                         concat_when_both=False, keep_raw=True):

    a = df[date_col1].astype(str).fillna("").replace("nan", "").str.strip()
    b = df[date_col2].astype(str).fillna("").replace("nan", "").str.strip()

    # build combined raw text
    if concat_when_both:
        combined_raw = a.copy()
        mask_a_empty = combined_raw == ""
        combined_raw[mask_a_empty] = b[mask_a_empty]
        mask_both = (a != "") & (b != "") & (a != b)
        combined_raw[mask_both] = a[mask_both] + "; " + b[mask_both]
    else:
        # prefer a, fall back to b
        combined_raw = a.where(a != "", b)

    if keep_raw:
        df[new_col + "_raw"] = combined_raw.replace("", pd.NA)

    # helper to extract a date-like substring and parse it
    date_patterns = [
        r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b',   # MM/DD/YYYY or MM-DD-YYYY
        r'\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b',     # YYYY/MM/DD or YYYY-MM-DD
        r'\b\w+\s\d{1,2},\s\d{4}\b',           # Month DD, YYYY
        r'\b\d{1,2}\s\w+\s\d{4}\b'             # DD Month YYYY
    ]

    def extract_and_parse(s):
        if s is None or s == "" or str(s).lower() == "nan":
            return pd.NaT
        for pat in date_patterns:
            m = re.search(pat, str(s))
            if m:
                # let pandas parse the matched substring
                return pd.to_datetime(m.group(0), errors="coerce")
        # fallback: try to parse whole string
        return pd.to_datetime(s, errors="coerce")

    df[new_col] = combined_raw.apply(extract_and_parse)
    df = df.drop(columns=[date_col2])

    return df

#petitions_2013_raw = combine_date_columns(petitions_2013_raw, "Date of Petition", "Column1",
#                                          new_col="Date of Petition", concat_when_both=True, keep_raw=True)
#petitions_2013_raw.head(3)

In [138]:
def load_all_datasets(file_pattern, folder_path):
    # build a search pattern inside the provided folder
    search_pattern = os.path.join(folder_path, file_pattern)
    all_files = glob.glob(search_pattern)
    df_list = []

    for filename in sorted(all_files):
        print(f"Reading: {filename}")
        try:
            df = pd.read_csv(filename)

            # extract year from filename 
            year_match = re.search(r'(\d{4})', os.path.basename(filename))
            if year_match:
                df['Source_Year'] = int(year_match.group(1))
            else:
                df['Source_Year'] = 'Unknown'

            

            # combine date columns for petitions/responses and responses if "Date Comments" exists
            if ("Date of Petition" in df.columns) and ("Date Comments" in df.columns):
                df = combine_date_columns(df, "Date of Petition", "Date Comments",
                                          new_col="Date of Petition", concat_when_both=True, keep_raw=True)
            elif ("Date of Petition" in df.columns) and ("Column1" in df.columns):
                df = combine_date_columns(df, "Date of Petition", "Column1",
                                          new_col="Date of Petition", concat_when_both=True, keep_raw=True)
            elif ("Date of Response" in df.columns) and ("Date Comments" in df.columns):
                df = combine_date_columns(df, "Date of Response", "Date Comments",
                                          new_col="Date of Response", concat_when_both=True, keep_raw=True)
            elif ("Date of Response" in df.columns) and ("Column1" in df.columns):
                df = combine_date_columns(df, "Date of Response", "Column1",
                                          new_col="Date of Response", concat_when_both=True, keep_raw=True)
            df_list.append(df)

        except Exception as e:
            print(f"Error reading {filename}: {e}")

    # concatenate and return a single df (or empty df if nothing loaded)
    if df_list:
        big_df = pd.concat(df_list, ignore_index=True)
        return big_df
    else:
        return pd.DataFrame()

petitions_path_pattern = "*PETITIONS*.csv"
responses_path_pattern = "*RESPONSES*.csv"

print("--- Loading All Petitions ---")
all_petitions_raw = load_all_datasets(petitions_path_pattern, "raw_petitions_data")

print("\n--- Loading All Responses ---")
all_responses_raw = load_all_datasets(responses_path_pattern, "raw_resp_data")

print("\nFinal Shapes:")
print("All Petitions shape:", all_petitions_raw.shape)
display(all_petitions_raw.head())

print("All Responses shape:", all_responses_raw.shape)
display(all_responses_raw.head())

--- Loading All Petitions ---
Reading: raw_petitions_data/2010_PETITIONS_COMPLETED_050125_CT_0725_MC.xlsx - Sheet1.csv
Reading: raw_petitions_data/2012_PETITIONS_COMPLETE_050325_MZ.xlsx - 2012.csv
Reading: raw_petitions_data/2013_PETITIONS_COMPLETE_050125_WZ_0725_EDITED BY MC.xlsx - 2013.csv
Reading: raw_petitions_data/2017_PETITIONS_COMPLETE_071825_MP.xlsx - Sheet1 (1).csv
Reading: raw_petitions_data/2018_PETITIONS_COMPLETE_070825_MP.xlsx - Sheet1.csv
Reading: raw_petitions_data/2019_PETITIONS_COMPLETE_050325_MZ_0725_REDONE BY MC.xlsx - Sheet1.csv
Reading: raw_petitions_data/2020_PETITIONS_COMPLETE_050125_WZ.xlsx - 2020.csv
Reading: raw_petitions_data/2024_PETITIONS_COMPLETE_050225_WZ.xlsx - 2024.csv

--- Loading All Responses ---
Reading: raw_resp_data/2010_RESPONSES_COMPLETED_050125CT_0725_MC.xlsx - 2010_RESPONSES_IN_PROGRESS_0501.csv
Reading: raw_resp_data/2013_RESPONSES_COMPLETE_050125_WZ_0725_EDITED BY MC.xlsx - 2013.csv
Reading: raw_resp_data/2014_RESPONSES_COMPLETE_071525_MP.xl

,File Name,Date of Petition,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Status of Review,Review Comments,Source_Year,Date of Petition_raw,Comments by RA,Date received
0,FDA-2010-P-0648-0008_Petition_for_Stay_of_Acti...,2019-10-07 00:00:00,G. Pohl-Boskamp GmbH & Company KG (“Pohl”),The petition was submitted through Hogan Lovel...,21 CFR 10.35 – Pertaining to petitions for sta...,"The September 20, 2013 approval of Abbreviated...",That the FDA stay the effective date of approv...,FDA approved Perrigo’s ANDA without addressing...,NaN,NaN,2010,"October 7, 2019",NaN,NaN
1,FDA-2010-P-0658-0001_Actavis_Elizabeth_LLC___C...,2010-12-29 00:00:00,Actavis Elizabeth LLC,No external representation is stated. The peti...,21 C.F.R. § 10.30 – Citizen petition procedure...,FDA's possible classification of LAMICTAL® (la...,Petitioner requests FDA to determine that LAMI...,"The product was approved on April 14, 2010, bu...",NaN,NaN,2010,"Printed December 29, 2010; Stamped December 30...",NaN,NaN
2,FDA-2010-P-0658-0003_Actavis_Elizabeth_LLC___W...,2011-06-06 00:00:00,Actavis Elizabeth LLC,The petition was directly submitted and withdr...,"Citizen Petition FDA-2010-P-0658-0001, origina...",The FDA’s determination regarding whether LAMI...,The petitioner originally requested the FDA to...,Actavis states that the innovator (SmithKline ...,NaN,NaN,2010,"Printed June 6, 2011; Stamped June 7, 2011",NaN,NaN
3,FDA-2010-P-0648-0001_G__Pohl_Boskamp_GmbH___Co...,2010-12-16 00:00:00,G. Pohl-Boskamp GmbH & Company KG \tNo law fir...,NaN,- 21 USC 355(b) and 355(g) – Drug approval and...,FDA's 2010 Draft Bioequivalence Recomm...,\tFDA should not approve generic nitroglycerin...,\t- Current BE guidance is inadequate for ensu...,NaN,NaN,2010,"December 16, 2010 (stamped received by FDA on ...",NaN,NaN
4,FDA-2010-P-0642-0005_Teva_Pharmaceutical_Indus...,2011-02-11 00:00:00,Teva Pharmaceutical Industries Ltd. and Teva N...,"Submitted directly by J. Michael Nicholas, Ph....",- 21 C.F.R. § 10.30 – Citizen petition process...,FDA’s approach to approving Abbreviated New Dr...,FDA should not approve any ANDA for a generic ...,- Copaxone®'s mechanism of action is not fully...,NaN,NaN,2010,"February 11, 2011 (date stamped by FDA upon re...",NaN,NaN


All Responses shape: (1342, 12)


,File Name,Date of Response,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Review Comments,Source_Year,Date of Response_raw,Dates Comments,Status of Review,Unnamed: 6
0,FDA-2003-P-0694-0004_Memorandum_for_the_Record...,2010-11-18 00:00:00,"Office of Nutrition, Labeling and Dietary Supp...",Closed\nThe petition was closed at the request...,"Section 403(i) of the Federal Food, Drug, and ...",The petition was voluntarily withdrawn by the ...,NaN,2010,"Stamped November 18, 2010 and November 20, 2010",NaN,NaN,NaN
1,FDA-2003-P-0694-0003_Letter_from_FDA_CFSAN_to_...,2003-07-21 00:00:00,Center for Food Safety and Applied Nutrition (...,No decision issued at the time\nFDA advised th...,"Section 403(i)(1) of the Federal Food, Drug, a...",FDA explained the delay was due to:\n\nThe age...,From 2003?,2010,"Stamped July 21, 2003 and July 22, 2003",NaN,NaN,NaN
2,FDA-2003-P-0563-0001_Memo_to_File_to_Administr...,2022-04-14 00:00:00,Center for Food Safety and Applied Nutrition (...,Administratively Closed\nThe petition was volu...,Section 10809 of the Farm Security and Investm...,The Farm Security and Investment Act of 2002 w...,NaN,2010,"April 14, 2022",NaN,NaN,NaN
3,FDA-2010-P-0012-0007_Alaunus_Pharmaceutical_LL...,2010-11-07 00:00:00,Division of Dockets Management,Withdrawn,Not Mentioned,Petition was withdrawn by Alaunus Pharmaceutic...,NaN,2010,"Printed November 7, 2010; Stamped November 8, ...",NaN,NaN,NaN
4,FDA-2010-P-0015-0006_Citizen_Petition_Denial_R...,2010-10-01 00:00:00,Center for Drug Evaluation and Research (CDER),Denied,21 C.F.R. § 10.30; Section 505(j)(2)(C) of the...,The proposed 7.5 mg strength is not supported ...,NaN,2010,"Stamped October 1, 2010 and October 17, 2010",NaN,NaN,NaN


#### Petitions
**DROP these columns from petitions:** 

['Status of Review', 'Review Comments', 'Comments by RA', 'Date received']
 
**KEEP these columns from petitons:**

['File Name', 'Date of Petition', 'Identity of Submitting Entity', 'Representation Details', 
'Cited Statutes or Regulations', 'FDA Action Commented On', 'Requested Action', 'Justification for Request', 'Source_Year']
___

#### Responses
**DROP these columns from responses:** 

['Review Comments', 'Dates Comments', 'Status of Review', 'Unnamed: 6']

**KEEP these columns from responses:**

['File Name', 'Date of Response', 'Responding FDA Center', 'Response to Petition',
'Cited Statutes or Regulations', 'Justification for Response', 'Source_Year']
___

## test code below to decide what to drop/keep

In [139]:
all_petitions_raw.columns

Index(['File Name', 'Date of Petition', 'Identity of Submitting Entity',
       'Representation Details', 'Cited Statutes or Regulations',
       'FDA Action Commented On', 'Requested Action',
       'Justification for Request', 'Status of Review', 'Review Comments',
       'Source_Year', 'Date of Petition_raw', 'Comments by RA',
       'Date received'],
      dtype='object')

In [140]:
all_responses_raw.columns

Index(['File Name', 'Date of Response', 'Responding FDA Center',
       'Response to Petition', 'Cited Statutes or Regulations',
       'Justification for Response', 'Review Comments', 'Source_Year',
       'Date of Response_raw', 'Dates Comments', 'Status of Review',
       'Unnamed: 6'],
      dtype='object')

In [141]:
# all_responses_raw[~all_responses_raw["Unnamed: 6"].isnull()]
all_petitions_raw['Date received'].unique()

array([nan])

In [142]:
# petitions with no date of petition
len(all_petitions_raw[all_petitions_raw["Date of Petition"].isnull()]) / len(all_petitions_raw)

0.01520387007601935

In [143]:
# petitions with no date of response
len(all_responses_raw[all_responses_raw["Date of Response"].isnull()]) / len(all_responses_raw)

0.039493293591654245

In [144]:
clean_stage_petitions_df = all_petitions_raw.drop(columns=['Status of Review', 'Review Comments', 'Comments by RA', 'Date received'])

clean_stage_resp_df = all_responses_raw.drop(columns=['Review Comments', 'Dates Comments', 'Status of Review', 'Unnamed: 6'])

# display(clean_stage_petitions_df.head(3))
# display(clean_stage_resp_df.head(3))

In [145]:
print("\nCleaned Shapes:")
print("Cleaned Petitions shape:", clean_stage_petitions_df.shape)
display(clean_stage_petitions_df.head(3))

print("Cleaned Responses shape:", clean_stage_resp_df.shape)
display(clean_stage_resp_df.head(3))


Cleaned Shapes:
Cleaned Petitions shape: (1447, 10)


,File Name,Date of Petition,Identity of Submitting Entity,Representation Details,Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Source_Year,Date of Petition_raw
0,FDA-2010-P-0648-0008_Petition_for_Stay_of_Acti...,2019-10-07 00:00:00,G. Pohl-Boskamp GmbH & Company KG (“Pohl”),The petition was submitted through Hogan Lovel...,21 CFR 10.35 – Pertaining to petitions for sta...,"The September 20, 2013 approval of Abbreviated...",That the FDA stay the effective date of approv...,FDA approved Perrigo’s ANDA without addressing...,2010,"October 7, 2019"
1,FDA-2010-P-0658-0001_Actavis_Elizabeth_LLC___C...,2010-12-29 00:00:00,Actavis Elizabeth LLC,No external representation is stated. The peti...,21 C.F.R. § 10.30 – Citizen petition procedure...,FDA's possible classification of LAMICTAL® (la...,Petitioner requests FDA to determine that LAMI...,"The product was approved on April 14, 2010, bu...",2010,"Printed December 29, 2010; Stamped December 30..."
2,FDA-2010-P-0658-0003_Actavis_Elizabeth_LLC___W...,2011-06-06 00:00:00,Actavis Elizabeth LLC,The petition was directly submitted and withdr...,"Citizen Petition FDA-2010-P-0658-0001, origina...",The FDA’s determination regarding whether LAMI...,The petitioner originally requested the FDA to...,Actavis states that the innovator (SmithKline ...,2010,"Printed June 6, 2011; Stamped June 7, 2011"


Cleaned Responses shape: (1342, 8)


,File Name,Date of Response,Responding FDA Center,Response to Petition,Cited Statutes or Regulations,Justification for Response,Source_Year,Date of Response_raw
0,FDA-2003-P-0694-0004_Memorandum_for_the_Record...,2010-11-18 00:00:00,"Office of Nutrition, Labeling and Dietary Supp...",Closed\nThe petition was closed at the request...,"Section 403(i) of the Federal Food, Drug, and ...",The petition was voluntarily withdrawn by the ...,2010,"Stamped November 18, 2010 and November 20, 2010"
1,FDA-2003-P-0694-0003_Letter_from_FDA_CFSAN_to_...,2003-07-21 00:00:00,Center for Food Safety and Applied Nutrition (...,No decision issued at the time\nFDA advised th...,"Section 403(i)(1) of the Federal Food, Drug, a...",FDA explained the delay was due to:\n\nThe age...,2010,"Stamped July 21, 2003 and July 22, 2003"
2,FDA-2003-P-0563-0001_Memo_to_File_to_Administr...,2022-04-14 00:00:00,Center for Food Safety and Applied Nutrition (...,Administratively Closed\nThe petition was volu...,Section 10809 of the Farm Security and Investm...,The Farm Security and Investment Act of 2002 w...,2010,"April 14, 2022"


In [146]:
# cleaning pharmaceutical companies corpus

# drop last row bc NaN
pharma_comps = pharma_comps.drop(len(pharma_comps) - 1)

# remove inc, lp, etc
def normalize_company_name(name):
    name_lower = name.lower()
    
    # looks for common suffixes (inc, llc, etc.) and removes them from the end of the comp name
    name_lower = re.sub(r'(\s*,\s*|\s+)(inc|llc|l\.p|lp|limited|corp|co|ltd)\.?\s*$', '', name_lower)
    
    # remove any remaining punctuation and extra whitespace
    name_lower = re.sub(r'[^\w\s]', '', name_lower)
    name_lower = name_lower.strip()
    
    return name_lower
    
pharma_names_list = pharma_comps["Sponsor"].str.lower().str.strip().sort_values().unique()
#display(pharma_names_list)

pharma_names_norm = pharma_comps["Sponsor"].apply(normalize_company_name).sort_values().unique()
#display(pharma_names_norm.head)

## Clean Petitions Data

### 1. Categorize Submitter

In [147]:
# cleaning "Identity of Submitting Entity", aka making a "Submitter" column

# remove rows w/ no submitting entity
clean_stage_petitions_df = clean_stage_petitions_df.dropna(subset = ["Identity of Submitting Entity"])

# removing "\n", "\t", "\n-", "\n•\u202f"
clean_stage_petitions_df["Submitter"] = clean_stage_petitions_df["Identity of Submitting Entity"].astype(str)
clean_stage_petitions_df["Submitter"] = clean_stage_petitions_df["Submitter"].str.strip().str.replace("\n-", " ").str.replace("\n•\u202f", " ").str.replace("\u202f", " ")


In [148]:
# categorize each submitter
individual_terms = ['individual', 'citizen', 'individual citizen', 'multiple co-signatories', ' md', ' m.d', ' m.d.', 
                    ' jd', ' j.d', ' j.d.', ' pharmd', ' pharm.d', ' pharm.d.', 'phd',  'ph.d', ' senator', 'senate', 'dr.',
                    'client', 'governor', 'petitioner)', 'on behalf of', 'esq', 'esq.', 'petitioner', 'dds', 'rac', 
                    'group of concerned', 'founder of the', 'physician assistant']
indus_corp_terms = ['llc', 'inc', 'inc.', 'pharma', 'ltd', 'limited', 'corp', 'laboratories', 'technologies', 'gmbh', 'kg', 
                    's.a.', 's.l.', 'medical care', 'holdings', 'group of companies', 'international', 'company']
outlier_indus = ['unilever']
law_consult_terms = ['llp', 'p.c.', 'p.c', "pc", 'law', 'legal', 'consultant', 'consulting', 'regulatory', 'associates', 'solution']
advoc_academic_terms = ['association', 'university', 'center for', 'society', 'foundation', 'research group', 'research', 
                        'lab', 'registry', 'institute', 'council', 'choice', "advocate", 'college', 'initiative', 'committee', 
                        'federation', 'alliance', 'defense', 'guild', 'task force', 'clinic', 'league', 'action', 'therapy', 
                        'study group', 'network mothers', 'board of directors', 'working group', 'patients first', 
                        'ethical treatment', 'board', 'dispensers', 'asthma', 'peta', 'hospital', 'clinic', 
                        'center for science', 'research institute']

def categorize_submitter(submitter_title):
    submitter_title_lower = submitter_title.lower()
    #submitter_title_norm = normalize_company_name(submitter_title)
    
    if pd.isna(submitter_title) or "anonymous" in submitter_title_lower:
        return "other"
        
    elif submitter_title_lower in pharma_names_list or submitter_title_lower in pharma_names_norm:
        return "industry/corporation"
        
    # for each identifier word in individual, industry, etc, check if that word exists in the title
    if any(word in submitter_title_lower for word in law_consult_terms):
        return "law/consulting"
    
    elif any(word in submitter_title_lower for word in advoc_academic_terms):
        return "advocacy/academic"

    elif submitter_title_lower in outlier_indus or any(word in submitter_title_lower for word in indus_corp_terms):
        return "industry/corporation"        
        
    elif any(word in submitter_title_lower for word in individual_terms):
        return "individual"

    # check for short individual's names (usually first, middle, last)
    elif len(submitter_title_lower.split()) <= 3:
        return "individual"

    else:
        return "other"

In [149]:
clean_stage_petitions_df["Submitter Type"] = clean_stage_petitions_df["Submitter"].apply(categorize_submitter)
#clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "law/consulting"]

clean_stage_petitions_df["Submitter Type"].value_counts()

Submitter Type
industry/corporation    622
law/consulting          467
advocacy/academic       213
individual              136
other                     7
Name: count, dtype: int64

In [150]:
other_submitter = clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "other"][["Submitter", "Submitter Type"]]
other_submitter.to_csv("test_datasets_eda/other_submitters.csv")

In [151]:
indus_corp_submitter = clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "industry/corporation"][["Submitter", "Submitter Type"]]
indus_corp_submitter.to_csv("test_datasets_eda/indus_corp_submitters.csv")

In [152]:
law_consult_submitter = clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "law/consulting"][["Submitter", "Submitter Type"]]
law_consult_submitter.to_csv("test_datasets_eda/law_consult_submitters.csv")

In [153]:
indiv_submitter = clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "individual"][["Submitter", "Submitter Type"]]
indiv_submitter.to_csv("test_datasets_eda/individual_submitters.csv")

In [154]:
adv_aca_submitter = clean_stage_petitions_df[clean_stage_petitions_df["Submitter Type"] == "advocacy/academic"][["Submitter", "Submitter Type"]]
adv_aca_submitter.to_csv("test_datasets_eda/adv_aca_submitters.csv")

### 2. Making unique ids, labeling document type (og petit, reconsideration), boolean column for if petition was withdrawn

In [155]:
# use numbers after "FDA-2017-P-..." as unique id -> make "File ID" column
clean_stage_petitions_df["File ID"] = clean_stage_petitions_df["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
clean_stage_petitions_df["File ID"] = clean_stage_petitions_df["File ID"].str.lower()
#display(clean_stage_petitions_df.head(3))

clean_stage_resp_df["File ID"] = clean_stage_resp_df["File Name"].str.extract(r'(FDA-\d{4}-P-\d{4})')
clean_stage_resp_df["File ID"] = clean_stage_resp_df["File ID"].str.lower()
#display(clean_stage_resp_df.head(3))

In [156]:
clean_stage_petitions_df.columns

Index(['File Name', 'Date of Petition', 'Identity of Submitting Entity',
       'Representation Details', 'Cited Statutes or Regulations',
       'FDA Action Commented On', 'Requested Action',
       'Justification for Request', 'Source_Year', 'Date of Petition_raw',
       'Submitter', 'Submitter Type', 'File ID'],
      dtype='object')

In [157]:
# adding withdrawals as a feature (aka new column)
def withdrawn_petit(df):
    df["Withdrawal File"] = df["File Name"].str.lower().str.contains("withdraw")
    withdrawals_id = df[df["Withdrawal File"] == True]["File ID"]
    df["Was_Withdrawn"] = df["File ID"].isin(withdrawals_id)
    #df.drop("Withdrawal File", axis = "columns", inplace = True)

# adding a column for document type (reconsideration, og petition, stay request)
def get_doc_type(row):
    file_name_lower = str(row["File Name"]).lower()
    
    if "reconsideration" in file_name_lower:
        return "reconsideration"
    elif "stay" in file_name_lower:
        return "stay request"
    return "original petition"

withdrawn_petit(clean_stage_petitions_df)
clean_stage_petitions_df["Document Type"] = clean_stage_petitions_df.apply(get_doc_type, axis = 1)

# cleaning
new_clean_petitions = clean_stage_petitions_df[clean_stage_petitions_df["Withdrawal File"] == False].copy()
new_clean_petitions = new_clean_petitions.drop(["Identity of Submitting Entity", "Withdrawal File", "Representation Details"], axis = "columns")
new_clean_petitions = new_clean_petitions.rename(columns = {"Cited Statutes or Regulations": "Petition Cited Statutes or Regulations", 
                                                            "File Name": "Petition File Name",
                                                            "Source_Year": "Petition Source Year"})
new_column_order = ['File ID', 'Petition File Name', 'Date of Petition', 'Petition Source Year', 'Was_Withdrawn', 'Document Type', 'Submitter', 'Submitter Type',  
                    'Petition Cited Statutes or Regulations', 'FDA Action Commented On', 'Requested Action', 'Justification for Request']

new_clean_petitions = new_clean_petitions[new_column_order]

# new_clean_petitions.to_csv("test_datasets_eda/new_clean_petitions.csv", index=False)

# new_clean_petitions.head(3)
display("Count of petitions that was withdrawn by petitioners: " + str(len(new_clean_petitions[new_clean_petitions["Was_Withdrawn"] == True]))) # count of petitions that had withdrawal letter
display("Proportion of petitions that was withdrawn by petitioners: " + str(len(new_clean_petitions[new_clean_petitions["Was_Withdrawn"] == True]) / len(new_clean_petitions))) # proportion

# drop petitions that are stay requests and was_withdrawn == True (withdrawn by petitioner)
new_clean_petitions = new_clean_petitions[~(new_clean_petitions["Document Type"] == "stay request") & (new_clean_petitions["Was_Withdrawn"] == False)].reset_index(drop=True)

'Count of petitions that was withdrawn by petitioners: 95'

'Proportion of petitions that was withdrawn by petitioners: 0.07031828275351591'

### 3. Merging Petitions and Responses
making sure the date of petition (og, recon) matches w/ responses (og, recon)

In [158]:
long_df = pd.merge(
    new_clean_petitions,
    clean_stage_resp_df,
    on = "File ID",
    how = "left"
)
def find_correct_pairs(group):
    # (parameter) group: subset dataframe of long_df (merged); this subset df is long_df grouped by file id and date of petition 
    
    def filter_helper(df, date, column):
        result = None

        # check if date exists
        if pd.notna(date) and not df.empty:
            matching = df.loc[df["Date of Response"] == date]
        
            if not matching.empty:
                result = matching[column].iloc[0]

        return result
    
    
    # # get a subset of original petit and reconsideration
    # petition_date = group["Date of Petition"].iloc[0]
    # valid_resp = group[pd.to_datetime(group["Date of Response"], errors="coerce") > pd.to_datetime(petition_date, errors="coerce")]

    # # within the valid responses, find the earliest interim
    # interims = valid_resp[valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    # earliest_interim_date = interims["Date of Response"].min() 

    # # within the valid responses, find the latest FINAL decision (non-interim)
    # finals = valid_resp[~valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    # latest_final_date = finals["Date of Response"].max() 

    # fixing code logic above:

    petition_date = group["Date of Petition"].iloc[0]
    # keep responses that happened AFTER this specific petition was filed
    valid_resp = group[group["Date of Response"] > petition_date] 
    
    # separate responses
    interims = valid_resp[valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    finals = valid_resp[~valid_resp["Response to Petition"].str.lower().str.contains("interim", na=False)]
    
    # filter out responses that shouldn't count as a "final" decision on the petition itself
    finals_no_stay = finals[~finals['Response to Petition'].str.lower().str.contains("stay|reconsider", na=False)]
    
    # the fixed logic: depends on whether its an original or a reconsideration
    is_original = group["Document Type"].iloc[0].lower() == 'original petition'
    
    if is_original:
        # og petitions: clock stops at the FIRST final decision (MIN)
        latest_final_date = finals_no_stay["Date of Response"].min()
    else:
        # reconsiderations: want the LAST decision on the appeal (MAX)
        latest_final_date = finals_no_stay["Date of Response"].max()

    earliest_interim_date = interims["Date of Response"].min() 
    interim_counts = len(interims)
    
    # get the final decision w/ that (above ^) latest final date
    final_decision_text = filter_helper(finals, latest_final_date, "Response to Petition")

    # count of interim responses for each file id
    interim_counts = len(interims)

    # responding FDA center
    fda_center = filter_helper(finals, latest_final_date, "Responding FDA Center")

    # cited statutes or regulations (ignoring interim)
    statutes_regs = filter_helper(finals, latest_final_date, "Cited Statutes or Regulations")

    # justification for interim response 
    justif_interim = filter_helper(interims, earliest_interim_date, "Justification for Response")

    # justification for final response
    justif_final = filter_helper(finals, latest_final_date, "Justification for Response")


    # return a series w/ the results for the specific group
    return pd.Series({
        "Interim Response Date": earliest_interim_date,
        "Final Decision": final_decision_text,
        "Last Response Date": latest_final_date,
        "Count of Interim": interim_counts,
        "Responding FDA Center (of Final Decision)": fda_center,
        "Responses Cited Statutes or Regulations": statutes_regs,
        "Justification for Interim Response": justif_interim,
        "Justification for Final Response": justif_final
    })

aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()
final_df = pd.merge(
    new_clean_petitions,
    aggregated_responses,
    on=["File ID", "Date of Petition"],
    how="left"
)

display(final_df.head(3)) 

# check for correctness
recon_petit_id = new_clean_petitions[new_clean_petitions["Petition File Name"].str.lower().str.contains("reconsider", regex=False)]
#display(final_df[final_df["File ID"].isin(recon_petit_id["File ID"])])


/var/folders/w1/yj3_r24s4tl7vlj_flvb3ntm0000gn/T/ipykernel_26982/234078908.py:92: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  aggregated_responses = long_df.groupby(["File ID", "Date of Petition"]).apply(find_correct_pairs).reset_index()


,File ID,Petition File Name,Date of Petition,Petition Source Year,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,Requested Action,Justification for Request,Interim Response Date,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response
0,fda-2010-p-0648,FDA-2010-P-0648-0001_G__Pohl_Boskamp_GmbH___Co...,2010-12-16 00:00:00,2010,False,original petition,G. Pohl-Boskamp GmbH & Company KG \tNo law fir...,law/consulting,- 21 USC 355(b) and 355(g) – Drug approval and...,FDA's 2010 Draft Bioequivalence Recomm...,\tFDA should not approve generic nitroglycerin...,\t- Current BE guidance is inadequate for ensu...,2011-06-16 00:00:00,Partially Granted and Partially Denied\n– The ...,2013-10-31 00:00:00,1.0,Center for Drug Evaluation and Research (CDER),"Section 505(j) of the Federal Food, Drug, and ...",The FDA states it has not resolved the issues ...,Scientific Basis: FDA carefully reviewed the p...
1,fda-2010-p-0642,FDA-2010-P-0642-0005_Teva_Pharmaceutical_Indus...,2011-02-11 00:00:00,2010,False,original petition,Teva Pharmaceutical Industries Ltd. and Teva N...,industry/corporation,- 21 C.F.R. § 10.30 – Citizen petition process...,FDA’s approach to approving Abbreviated New Dr...,FDA should not approve any ANDA for a generic ...,- Copaxone®'s mechanism of action is not fully...,NaN,Denied\nThe FDA formally denied the citizen pe...,2011-06-08 00:00:00,0.0,Center for Drug Evaluation and Research (CDER)...,Section 505(q) of the FDCA (21 U.S.C. § 355)\n...,None,FDA emphasized its broad discretion under the ...
2,fda-2010-p-0011,FDA-2010-P-0011-0001_Bright_Future_Pharmaceuti...,2010-01-04 00:00:00,2010,False,original petition,Bright Future Pharmaceutical Lab. Ltd.,advocacy/academic,Section 505(j)(2)(C) of the FDCA; 21 CFR 25.300,Request to designate Panadol® as RLD or advise...,Permit submission of ANDA for BF-PARADAC (500m...,BF-PARADAC has been marketed in Hong Kong sinc...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


### 4. Clean "Final Decision" Column

In [159]:
final_df["Final Decision"].str.contains("partial")

0       False
1       False
2         NaN
3       False
4       False
        ...  
1244     None
1245    False
1246    False
1247    False
1248    False
Name: Final Decision, Length: 1249, dtype: object

In [160]:
# clean up "Final Decision" column
def clean_decision(decision):
    if pd.isna(decision):
        return "no decision"
    
    decision_string_lower = str(decision).lower() 

    # "partially denied" vs "Partially Approved / Partially Denied"
    if (("partially" in decision_string_lower)):
        return "partially approved / denied"
    
    # "approved" vs "approved (Determination Issued)"
    elif "approved" in decision_string_lower or "granted" in decision_string_lower:
        return "approved"
        
    # other more straightforward decisions
    elif "denied" in decision_string_lower:
        return "denied"
    elif "dismissed" in decision_string_lower:
        return "dismissed (moot)"
    elif "withdrawn" in decision_string_lower:
        return "withdrawn"
        
    return "other"


# make new column for cleaned final decision  
final_df["Cleaned Final Decision"] = final_df["Final Decision"].apply(clean_decision)
final_df["Cleaned Final Decision"].value_counts()

Cleaned Final Decision
no decision                    672
denied                         247
approved                       177
other                           58
partially approved / denied     46
dismissed (moot)                33
withdrawn                       16
Name: count, dtype: int64

In [161]:
# check other category
# final_df[final_df["Cleaned Final Decision"] == "other"]

### 5. Calculate "Response Time (Days)"

In [162]:
# calculate response time
def calc_resp_time_and_got_decision(dataset):
    # column to denote if the data got final decision
    dataset["Received Final Decision"] = ~pd.isna(dataset["Last Response Date"])

    # impute last response date for censored data & calculate response time
    dataset["Last Response Date"] = dataset["Last Response Date"].fillna(pd.to_datetime("2025-10-15"))
    dataset["Date of Petition"] = pd.to_datetime(dataset["Date of Petition"], errors="coerce")
    dataset["Last Response Date"] = pd.to_datetime(dataset["Last Response Date"], errors="coerce")
    dataset["Response Time (Days)"] = (dataset["Last Response Date"] - dataset["Date of Petition"]).dt.days

calc_resp_time_and_got_decision(final_df)
display(final_df.head(3))

,File ID,Petition File Name,Date of Petition,Petition Source Year,Was_Withdrawn,Document Type,Submitter,Submitter Type,Petition Cited Statutes or Regulations,FDA Action Commented On,...,Final Decision,Last Response Date,Count of Interim,Responding FDA Center (of Final Decision),Responses Cited Statutes or Regulations,Justification for Interim Response,Justification for Final Response,Cleaned Final Decision,Received Final Decision,Response Time (Days)
0,fda-2010-p-0648,FDA-2010-P-0648-0001_G__Pohl_Boskamp_GmbH___Co...,2010-12-16,2010,False,original petition,G. Pohl-Boskamp GmbH & Company KG \tNo law fir...,law/consulting,- 21 USC 355(b) and 355(g) – Drug approval and...,FDA's 2010 Draft Bioequivalence Recomm...,...,Partially Granted and Partially Denied\n– The ...,2013-10-31,1.0,Center for Drug Evaluation and Research (CDER),"Section 505(j) of the Federal Food, Drug, and ...",The FDA states it has not resolved the issues ...,Scientific Basis: FDA carefully reviewed the p...,partially approved / denied,True,1050.0
1,fda-2010-p-0642,FDA-2010-P-0642-0005_Teva_Pharmaceutical_Indus...,2011-02-11,2010,False,original petition,Teva Pharmaceutical Industries Ltd. and Teva N...,industry/corporation,- 21 C.F.R. § 10.30 – Citizen petition process...,FDA’s approach to approving Abbreviated New Dr...,...,Denied\nThe FDA formally denied the citizen pe...,2011-06-08,0.0,Center for Drug Evaluation and Research (CDER)...,Section 505(q) of the FDCA (21 U.S.C. § 355)\n...,None,FDA emphasized its broad discretion under the ...,denied,True,117.0
2,fda-2010-p-0011,FDA-2010-P-0011-0001_Bright_Future_Pharmaceuti...,2010-01-04,2010,False,original petition,Bright Future Pharmaceutical Lab. Ltd.,advocacy/academic,Section 505(j)(2)(C) of the FDCA; 21 CFR 25.300,Request to designate Panadol® as RLD or advise...,...,NaN,2025-10-15,0.0,NaN,NaN,NaN,NaN,no decision,False,5763.0


### Clean "Responding FDA Center (of Final Decision)"

In [163]:
# findings for this categorization that needs fixing:
# Cleaned FDA Responding Center
# pending    672
# CDER       404
# CDRH        58
# CFSAN       33
# CVM         28
# DMS         26
# CBER        13
# offices      6
# other        5
# CTP          4
# Name: count, dtype: int64

# this makes the HR absurdly high for like HR = 48.55 for DMS -> 
# indicates numerical instability caused by a phenomenon called complete separation.
# what this means: the 28 petitions reviewed by DMS likely ALL received a final decision (event=1), and/or had a very short duration ->
# model coefficient is trying to go to infinity to perfectly explain this finding, which is a sign of an unreliable estimate

#----------------------------------------------------------------------------------------------------------------------------------------------
# remove inc, lp, etc
def clean_resp_fda(name):
    if pd.isna(name):
        return "pending"
    
    name_lower = name.lower().strip()
    
    # WHEN YOU ADD THE NEW DATASETS FOR DIFFERENT YEAR: might need to add more centers!
    if "center for drug evaluation and research" in name_lower or "cder" in name_lower:
        return "CDER"
    elif "center for devices and radiological health" in name_lower or "cdrh" in name_lower:
        return "CDRH"
    elif "center for biologics evaluation and research" in name_lower or "cber" in name_lower:
        return "CBER"
    elif "center for food safety and applied nutrition" in name_lower:
        return "CFSAN"
    elif "center for veterinary medicine" in name_lower or "cvm" in name_lower:
        return "CVM"
    elif "center for tobacco products" in name_lower or "ctp" in name_lower:
        return "CTP"
    elif "dockets management" in name_lower or "dms" in name_lower:
        return "DMS"
    # elif "office of regulatory policy" in name_lower or "orp" in name_lower:
    #     return "ORP"
    # elif "office of the commissioner" in name_lower or "oc" in name_lower:
    #     return "OC"
    # elif "office of the chief scientist" in name_lower or "ocs" in name_lower:
    #     return "OCS"
    # elif "office of regulatory affairs" in name_lower or "ora" in name_lower:
    #     return "ORA"
    elif "office" in name_lower:
        return "offices"
    else:
        return "other"
    
final_df["Cleaned FDA Responding Center"] = final_df["Responding FDA Center (of Final Decision)"].apply(clean_resp_fda)

# fix issue above by merging lower count centers into "other" to avoid complete separation issue
low_count_categories = ['CBER', 'CFSAN', 'CVM', 'DMS', 'offices', 'other', 'CTP']
final_df['Responding Center Grouped'] = np.where(
    final_df['Cleaned FDA Responding Center'].isin(low_count_categories),
    'Other/Specialty Center',
    final_df['Cleaned FDA Responding Center']
)

display(final_df["Cleaned FDA Responding Center"].value_counts())
display(final_df["Responding Center Grouped"].value_counts())

Cleaned FDA Responding Center
pending    672
CDER       404
CDRH        58
CFSAN       33
CVM         28
DMS         26
CBER        13
offices      6
other        5
CTP          4
Name: count, dtype: int64

Responding Center Grouped
pending                   672
CDER                      404
Other/Specialty Center    115
CDRH                       58
Name: count, dtype: int64

In [164]:
final_df.columns

Index(['File ID', 'Petition File Name', 'Date of Petition',
       'Petition Source Year', 'Was_Withdrawn', 'Document Type', 'Submitter',
       'Submitter Type', 'Petition Cited Statutes or Regulations',
       'FDA Action Commented On', 'Requested Action',
       'Justification for Request', 'Interim Response Date', 'Final Decision',
       'Last Response Date', 'Count of Interim',
       'Responding FDA Center (of Final Decision)',
       'Responses Cited Statutes or Regulations',
       'Justification for Interim Response',
       'Justification for Final Response', 'Cleaned Final Decision',
       'Received Final Decision', 'Response Time (Days)',
       'Cleaned FDA Responding Center', 'Responding Center Grouped'],
      dtype='object')

In [165]:
final_df.to_csv("cleaned_all_years_data.csv", index=False)

In [175]:
len(final_df[(final_df["Cleaned Final Decision"] == "no decision")]) / len(final_df) 

0.5380304243394716

In [174]:
len(final_df[(final_df["Count of Interim"] == 0) & (final_df["Cleaned Final Decision"] != "no decision")]) / len(final_df) 

0.3018414731785428

In [170]:
final_df[final_df["Response Time (Days)"] > 180][["File ID", "Date of Petition", "Last Response Date", "Response Time (Days)", "Cleaned Final Decision"]]

,File ID,Date of Petition,Last Response Date,Response Time (Days),Cleaned Final Decision
0,fda-2010-p-0648,2010-12-16,2013-10-31,1050.0,partially approved / denied
2,fda-2010-p-0011,2010-01-04,2025-10-15,5763.0,no decision
3,fda-2010-p-0012,2009-12-21,2010-11-07,321.0,withdrawn
4,fda-2010-p-0015,2010-01-05,2010-10-01,269.0,denied
5,fda-2010-p-0016,2009-12-31,2025-10-15,5767.0,no decision
...,...,...,...,...,...
1237,fda-2024-p-3761,2024-07-31,2025-10-15,441.0,no decision
1239,fda-2024-p-3820,2024-08-09,2025-10-15,432.0,no decision
1241,fda-2024-p-3822,2024-08-10,2025-10-15,431.0,no decision
1243,fda-2024-p-3905,2024-08-15,2025-02-12,181.0,approved
